In [20]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
from langchain_pinecone import PineconeVectorStore
import os
from pinecone import Pinecone, ServerlessSpec
from transformers import pipeline
from langchain_ollama.llms import OllamaLLM

In [3]:
load_dotenv()

True

In [5]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
if not PINECONE_API_KEY:
    raise ValueError("Pinecone API key is missing. Please set the PINECONE_API_KEY in the .env file.")

In [8]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [12]:
index_name = "medical-chatbot"
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")

/tmp/ipykernel_51144/3125231043.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")


In [13]:
get_op = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
    )

In [14]:
get_op

In [15]:
retriever = get_op.as_retriever(search_type="similarity", search_kwards={"k":3})

In [23]:
llm = OllamaLLM(model="llama3.1")

In [24]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

In [32]:
def run_query(query):
    try:
        response = qa_chain.invoke(query)
        return response
    except Exception as e:
        print(f"Error running query: {e}")
        return None

In [35]:
query = "What is acne?"

In [36]:
response = run_query(query)
print(f"Response for '{query}': {response}")

Response for 'What is acne?': {'query': 'What is acne?', 'result': "I don't know. The provided context does not mention anything about acne. It discusses corticosteroids, skin conditions such as contact dermatitis and rash, and ulcers, but it does not provide information on acne."}
